## Analysing output of PrEWRunRK fits

Using output of test toy fits from PrEW, import it into the PrOut python classes and create some histograms with it.

### Import reader module

In [ ]:
import sys
sys.path.append("../source")
import DistrReader as DR

### Import other modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

### Set output directory

In [ ]:
output_dir = "../output/plots/"
Path(output_dir).mkdir(parents=True, exist_ok=True)

### Read distribution file

In [ ]:
dr = DR.DistrReader("../output/selection_result.out")
dr.read()
print("Found {} distribution(s).".format(len(dr.distrs)))

### Define cuts to study

In [ ]:
cut_values = [0,5,10,15,20]
colors = ["#AA0A3C", "#8214A0", "#006E82", "#005AC8", "#0A9B4B"]

### Plot cut influence on each distribution

In [ ]:
# Draw how differential distributions at different pol. configurations change for different individual bin cuts
for distr in dr.distrs:
    # Draw one for each axis
    for axis in range(distr.dim):
        fig_cut, ax_cut = plt.subplots(tight_layout=True)
        fig_cut.suptitle("{} {}".format(distr.name,distr.pol_config))
        
        # Get the projections for each cut, check if axis actually exists and draw if so
        axis_valid = True
        for i_cut in range(len(cut_values)):
            cut_value = cut_values[i_cut]
            x, y = distr.projection(axis,cut_value)
            # Check if axis actually exists
            if (len(x) == 1):
                axis_valid = False
                break
            dx = x[1] - x[0] # Assuming that steps are all equal
            x_edges = np.append(np.array([x])-0.5*dx, x[-1]+0.5*dx)
            
            # Histogram drawing for all-zeroes more compilated...
            if (np.amax(y) == 0) :
                ax_cut.hist([],bins=x_edges,color=colors[i_cut], histtype='step', label="{}".format(cut_value))
            else :
                ax_cut.hist(x,bins=x_edges,weights=y,color=colors[i_cut], histtype='step', label="{}".format(cut_value))
            
        if not axis_valid: continue # Distribution wasn't actually differential in this observable => Skip it
        
        ax_cut.legend(title="Cut value", loc=0)
        
        fig_cut.savefig("{}cuts_{}_{}_axis{}.pdf".format(output_dir,distr.name,distr.pol_config,axis))

### Plot overall cut influence
Collect influence of the cuts on the distribution integrals

In [ ]:
# First find all (relative) integrals and put them in an ordered map
distr_map = {}
for distr in dr.distrs:
    rel_integr_after_cuts = np.array([distr.integral(cut_value) for cut_value in cut_values]) / distr.integral()
    if not distr.pol_config in distr_map.keys():
        distr_map[distr.pol_config] = {distr.name : rel_integr_after_cuts}
    else:
        distr_map[distr.pol_config][distr.name] = rel_integr_after_cuts
        
# Plot cut influence for each polarisation configuration
for pol_config, distrs in distr_map.items():
    x_vals = np.arange(0.5,len(distrs)+0.5)
    x_names = []
    y_vals = np.empty(shape=(len(cut_values),len(distrs)))
    
    i_distr = 0
    for name, integrals in distrs.items():
        x_names.append(name)
        for i_cut in range(len(cut_values)):
            y_vals[i_cut][i_distr] = integrals[i_cut]#np.append(y_vals[i_cut],integrals[i_cut])
        i_distr += 1

    # Create the figure
    fig_int, ax_int = plt.subplots(tight_layout=True)
    fig_int.suptitle("{}".format(pol_config))
    ax_int.set_ylabel("Rel. integral after cut")
    
    # Plot all integrals for all cut values
    for i_cut in range(len(cut_values)):
        ax_int.errorbar(x_vals,y_vals[i_cut],xerr=0.25,label="{}".format(cut_values[i_cut]), ecolor=colors[i_cut], fmt='none')
    
    # Sensible axis limits
    ax_int.set_ylim(0,1.1)
    
    # We want to show all ticks...
    ax_int.set_xticks(np.arange(0.5,len(distrs)+0.5))
    # ... and label them with the respective list entries
    ax_int.set_xticklabels(x_names)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax_int.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
    
    # Create the legend
    plt.legend(title="Cut value", loc=0)
    
    # Save the figure
    fig_int.savefig("{}integrals_{}.pdf".format(output_dir,pol_config))